# CNN

In [1]:
import glob
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
IMG_DIM = (150, 150)
np.random.seed(42)

/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Tommy/opt/anaconda3/e

In [3]:
# load data
_data_path = '../datasets and figures/asset2/dog_cat/'


# _data_path = './asset2/dog_cat/'

train_dir = _data_path+"training_data/"
valid_dir = _data_path+"validation_data/"
test_dir = _data_path+"test_data/"

train_files = glob.glob(train_dir+"*") 
print(len(train_files))
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('\\')[-1].split('.')[0].strip() for fn in train_files]
print(train_imgs.shape)


3000
(3000, 150, 150, 3)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)

In [6]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Input
from keras.models import Sequential
from keras import optimizers
from keras.models import Model

Using TensorFlow backend.


### Cov2D takes the input_shape  as (150,150,3)

### Cov1D takes the input_shape as (1,150,3)

In [7]:
#build model
input_shape=(150,150,3)
img_input = Input(shape=input_shape)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(img_input)
x = MaxPooling2D((2, 2), padding='same')(x)
    
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
    
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
    
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x= Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dense(128,activation='relu')(x)
output = Dense(1,activation='sigmoid')(x)

model = Model(img_input, output)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 128)       73856     
_____________________________________________________

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])
# model.fit(train_imgs,train_labels_enc, steps_per_epoch=100, epochs=20,
#                               validation_data=val_generator, validation_steps=50, verbose=1)
model.fit(train_imgs, train_labels_enc, batch_size=225 ,epochs=3, validation_split=0.2 )

Instructions for updating:
Use tf.cast instead.
Train on 2400 samples, validate on 600 samples
Epoch 1/3
2400/2400 [==============================] - 81s 34ms/step - loss: 0.1544 - accuracy: 0.9658 - val_loss: 1.0000e-07 - val_accuracy: 1.0000
Epoch 2/3
2400/2400 [==============================] - 74s 31ms/step - loss: 1.0000e-07 - accuracy: 1.0000 - val_loss: 1.0000e-07 - val_accuracy: 1.0000
Epoch 3/3
2400/2400 [==============================] - 72s 30ms/step - loss: 1.0000e-07 - accuracy: 1.0000 - val_loss: 1.0000e-07 - val_accuracy: 1.0000


In [12]:
#Extract output of 'layer[10](dense_7)'
intermediate_layer_model = Model(inputs = model.input, outputs = model.layers[10].output)
intermediate_layer_output = intermediate_layer_model.predict(train_imgs)
print(type(intermediate_layer_output))



<class 'numpy.ndarray'>


In [13]:
print(model.layers[10].output)

Tensor("dense_1/Relu:0", shape=(?, 512), dtype=float32)


In [14]:
# Classification using kNN

train_x = intermediate_layer_output[0:2000]
train_label = train_labels_enc[0:2000]
test_x = intermediate_layer_output[2000:3000]
test_label = train_labels_enc[2000:3000]
from sklearn.neighbors import KNeighborsClassifier

knn_3 = KNeighborsClassifier(n_neighbors=3) # Testing with k = 3
knn_3.fit(train_x,train_label)
y_pred_knn = knn_3.predict(test_x)
score = knn_3.score(test_x,test_label)
print(score)

1.0


# AutoEncoder

In [13]:
import glob
import numpy as np
import os
import PIL
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from datetime import datetime
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten, Dense, UpSampling2D
from keras.models import *
from keras.optimizers import SGD,Adam,Adagrad,Nadam,RMSprop
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.models import Model

np.random.seed(42)

In [15]:
_data_path = '../datasets and figures/asset2/dog_cat/'

# _asset_path = 'asset2/dog_cat/'

train_dir = _asset_path+'training_data/'
from sklearn.preprocessing import LabelEncoder
import pandas as pd
IMG_DIM = (48, 48)


train_files = glob.glob(train_dir+"*") 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
print(train_imgs.shape)

(3000, 48, 48, 3)


In [15]:
input_shape=(48, 48, 3)      
img_input = Input(shape=input_shape)       
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(img_input, decoded)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 16)        9232      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 16)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 16)        0   

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy')
hist = model.fit(train_imgs, train_imgs,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_split=0.2)

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 21s 9ms/step - loss: -78.0071 - val_loss: -4328.5440
Epoch 2/5
2400/2400 [==============================] - 21s 9ms/step - loss: -436.2186 - val_loss: -9663.1027
Epoch 3/5
2400/2400 [==============================] - 21s 9ms/step - loss: -888.5355 - val_loss: -7873.5651
Epoch 4/5
2400/2400 [==============================] - 21s 9ms/step - loss: -1444.0336 - val_loss: -8248.3335
Epoch 5/5
2400/2400 [==============================] - 21s 9ms/step - loss: -2088.4288 - val_loss: -17288.4038


In [17]:
#Extract output of 'layer[3](max_pooling2d_24)'
intermediate_layer_model = Model(inputs = model.input, outputs = model.layers[3].output)
intermediate_layer_output = intermediate_layer_model.predict(train_imgs)
print(intermediate_layer_output.shape)

(3000, 24, 24, 16)


In [18]:
train_imgs = intermediate_layer_output.reshape((len(intermediate_layer_output), np.prod(intermediate_layer_output.shape[1:])))
print(train_imgs.shape)

(3000, 9216)


In [20]:
train_x = train_imgs[0:2000]
train_label = train_labels_enc[0:2000]
test_x = train_imgs[2000:3000]
test_label = train_labels_enc[2000:3000]
print(test_x.shape)
print(test_label.shape)
# LogisticRegression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

lr.fit(train_x,train_label)  ## decoded_imgs_train: (60000, 64) 

y_pred_lr = lr.predict(test_x)  ## encoded_imgs_test: (10000, 64)
score1 = lr.score(test_x,test_label)
# score2 = lr.score(decoded_imgs_train,train_labels)
print(score1)
# print(score2)

(1000, 9216)
(1000,)
0.383


C:\Users\xzhu2\Documents\ProgramData\anaconda3\envs\AML158736\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
